To demonstrate your familiarity with OpenAI API, and also Ollama, I build a tool that takes a technical question,  
and responds with an explanation.

In [25]:
# imports
import os
from dotenv import load_dotenv
import gradio as gr
from openai import OpenAI


In [26]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [27]:
# set up environment
load_dotenv(override=True)
openai = OpenAI()
api_key = os.getenv('OPENAI_API_KEY')

In [28]:
# prompts

system_prompt = "You are a helpful technical tutor who answers questions about any coding language, software engineering, data science and LLMs"
system_prompt += "If the the user is asking anything but the technical questions, you should politely decline to answer"
system_prompt += "mention that you are trained only to answer only the technical questions"


In [29]:
# Get response from either OpenAI or Ollama
def get_response(model_type, user_prompt):
    user_prompt = "Please give a detailed explanation to the following question: " + user_prompt
    messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
    ]
    
    if model_type == "OpenAI":
        openai.api_key = api_key
        response = openai.chat.completions.create(
            model=MODEL_GPT,  # You can change the model if needed
            messages=messages
        )
        return response.choices[0].message.content
    
    elif model_type == "Ollama":
        # Use Ollama via OpenAI API
        ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
        response = ollama_via_openai.chat.completions.create(
                    model=MODEL_LLAMA,
                    messages=messages
                )
        return response.choices[0].message.content

    
    else:
        return "Invalid model selected."

In [30]:
# Function to chat with the model
def chat_with_model(model, prompt):
    return get_response(model, prompt)

# Gradio interface
gr.Interface(
    fn=chat_with_model,
    inputs=[
        gr.Dropdown(choices=["OpenAI", "Ollama"], label="Choose Model"),
        gr.Textbox(lines=4, placeholder="Enter your prompt here...", label="Prompt")
    ],
    outputs=gr.Markdown(label="Response"),
    title="EXPLAIN IT!",
    description="Ask any technical question and get it explained by AI model.",
    
    allow_flagging="never"
).launch(share=True)

/Users/virensasalu/miniconda3/envs/llms/lib/python3.11/site-packages/gradio/interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7881
* Running on public URL: https://7713c1dc33a7ddb452.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
